In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import (
    HumanMessage, 
    SystemMessage, 
    ToolMessage
)
from langchain.tools import tool
from langchain_core.output_parsers.openai_tools import parse_tool_calls
from dotenv import load_dotenv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import beta

In [2]:
load_dotenv()

True

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    base_url="https://openai.vocareum.com/v1",
    api_key=os.getenv("VOCAREUM_OPENAI_API_KEY")
)

**Tool creation**

In [ ]:
@tool
def plot_beta_distribution(alpha: float, beta_param: float, n_points: int = 1000):
    """
    Plots the Beta distribution for given alpha and beta parameters.
    
    Args:
        alpha (float): Shape parameter α (> 0)
        beta_param (float): Shape parameter β (> 0)
        n_points (int): Number of points in the x-axis grid
    """
    # Create x values between 0 and 1
    x = np.linspace(0, 1, n_points)
    
    # Compute the Beta PDF for each x
    y = beta.pdf(x, alpha, beta_param)
    
    # Plot
    plt.figure(figsize=(8, 4))
    plt.plot(x, y, 'b-', lw=2, label=f'Beta(α={alpha}, β={beta_param})')
    plt.title("Beta Distribution", fontsize=14)
    plt.xlabel("x", fontsize=12)
    plt.ylabel("Density", fontsize=12)
    plt.legend()
    plt.grid(alpha=0.3)
    plt.show()
    
    return f"Plotted Beta distribution with α={alpha}, β={beta_param}"

In [5]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [6]:
tools = [multiply, plot_beta_distribution]
tool_map = {tool.name:tool for tool in tools}

In [7]:
tool_map

{'multiply': StructuredTool(name='multiply', description='Multiply two numbers.', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x11bc1f9c0>),
 'plot_beta_distribution': StructuredTool(name='plot_beta_distribution', description='Plots the Beta distribution for given alpha and beta parameters.\n\nArgs:\n    alpha (float): Shape parameter α (> 0)\n    beta_param (float): Shape parameter β (> 0)\n    n_points (int): Number of points in the x-axis grid', args_schema=<class 'langchain_core.utils.pydantic.plot_beta_distribution'>, func=<function plot_beta_distribution at 0x11bc072e0>)}

**Binding Tools**

In [16]:
llm_with_tools = llm.bind_tools(tools, tool_choice="any")

In [17]:
question = "what does a beta distribution look like with alpha=2 and beta=5?"

In [18]:
print(question)

what does a beta distribution look like with alpha=2 and beta=5?


In [19]:
messages = [
    SystemMessage("You're a helpful assistant. ALWAYS use the tools provided to you. NEVER calculate the answer yourself."),
    HumanMessage(question)
]

In [20]:
ai_message = llm_with_tools.invoke(messages)

In [21]:
# Option 1: Use tool_calls attribute directly (preferred)
if ai_message.tool_calls:
    for tool_call in ai_message.tool_calls:
        # tool_call is already parsed!
        function_name = tool_call['name']
        arguments = tool_call['args']
        print(function_name, arguments)
        print(ai_message.additional_kwargs)
else:
    print(f"No tool calls. Answer: {ai_message.content}")

plot_beta_distribution {'alpha': 2, 'beta_param': 5}
{'refusal': None}


In [22]:
for a in ai_message:
    print(a)

('content', '')
('additional_kwargs', {'refusal': None})
('response_metadata', {'token_usage': {'completion_tokens': 20, 'prompt_tokens': 163, 'total_tokens': 183, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CaoUhljvWpqsUg59cD4UsgrtsS2mr', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None})
('type', 'ai')
('name', None)
('id', 'lc_run--0bbccafe-414a-44ed-b5a9-394089ace232-0')
('tool_calls', [{'name': 'plot_beta_distribution', 'args': {'alpha': 2, 'beta_param': 5}, 'id': 'call_j9ir6kphn7qHEkeRNXhRntvl', 'type': 'tool_call'}])
('invalid_tool_calls', [])
('usage_metadata', {'input_tokens': 163, 'output_tokens': 20, 'total_tokens': 183, 'input_token_details': {'audio': 0, 'cache_read':

In [23]:
ai_message

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 163, 'total_tokens': 183, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CaoUhljvWpqsUg59cD4UsgrtsS2mr', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--0bbccafe-414a-44ed-b5a9-394089ace232-0', tool_calls=[{'name': 'plot_beta_distribution', 'args': {'alpha': 2, 'beta_param': 5}, 'id': 'call_j9ir6kphn7qHEkeRNXhRntvl', 'type': 'tool_call'}], usage_metadata={'input_tokens': 163, 'output_tokens': 20, 'total_tokens': 183, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [24]:
messages.append(ai_message)

In [25]:
messages

[SystemMessage(content="You're a helpful assistant. ALWAYS use the tools provided to you. NEVER calculate the answer yourself.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what does a beta distribution look like with alpha=2 and beta=5?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 163, 'total_tokens': 183, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CaoUhljvWpqsUg59cD4UsgrtsS2mr', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--0bbccafe-414a-44ed-b5a9-394089ace232-0', tool_calls=[{'name': 'plot_beta_distr

In [26]:
for m in messages:
    print(m)

content="You're a helpful assistant. ALWAYS use the tools provided to you. NEVER calculate the answer yourself." additional_kwargs={} response_metadata={}
content='what does a beta distribution look like with alpha=2 and beta=5?' additional_kwargs={} response_metadata={}
content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 163, 'total_tokens': 183, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CaoUhljvWpqsUg59cD4UsgrtsS2mr', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--0bbccafe-414a-44ed-b5a9-394089ace232-0' tool_calls=[{'name': 'plot_beta_distribution', 'args': {'alpha': 2, 'beta_param': 5}, 'id

**Using Tool Calls**

In [ ]:
# Use tool_calls directly - they're already parsed!
if ai_message.tool_calls:
    parsed_tool_calls = ai_message.tool_calls
    print(f"Found {len(parsed_tool_calls)} tool call(s)")
else:
    parsed_tool_calls = []
    print("No tool calls found")

In [42]:
parsed_tool_calls

NameError: name 'parsed_tool_calls' is not defined

In [ ]:
for tool_call in parsed_tool_calls:
    tool_call_id = tool_call['id']
    function_name = tool_call['name']
    arguments = tool_call['args']
    
    print(f"Executing: {function_name}({arguments})")
    
    func = tool_map[function_name]
    result = func.invoke(arguments)
    
    # Convert result to string if needed
    result_str = str(result)
    
    tool_message = ToolMessage(
        content=result_str,
        name=function_name,
        tool_call_id=tool_call_id,
    )
    messages.append(tool_message)
    
print(f"Executed {len(parsed_tool_calls)} tool(s)")

**Sending the result back to the LLM**

In [44]:
messages

[SystemMessage(content="You're a helpful assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what does a beta distribution look like with alpha=2 and beta=5?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 149, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CaoSIGh2rT2P8twLi54FovjzvOV8R', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--3e280dc7-983e-41fc-8bc4-db0d432ae628-0', tool_calls=[{'name': 'plot_beta_distribution', 'args': {'alpha': 2, 'beta_param': 5}, 'id': 'call_pfjViHWIX7uDvA7

In [ ]:
# Send messages back to LLM to get final formatted response
final_response = llm_with_tools.invoke(messages)
print("Final Response:")
print(final_response.content)

In [46]:
ai_message

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 149, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CaoSIGh2rT2P8twLi54FovjzvOV8R', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--3e280dc7-983e-41fc-8bc4-db0d432ae628-0', tool_calls=[{'name': 'plot_beta_distribution', 'args': {'alpha': 2, 'beta_param': 5}, 'id': 'call_pfjViHWIX7uDvA7BSWAkfHyU', 'type': 'tool_call'}], usage_metadata={'input_tokens': 149, 'output_tokens': 20, 'total_tokens': 169, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [47]:
question = "Plot the Beta distribution for alpha=2 and beta=5"
messages = [
    SystemMessage("You're a helpful assistant"),
    HumanMessage(question)
]

ai_message = llm_with_tools.invoke(messages)
print(ai_message)

content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 145, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CaoTOdB9cgAz1BWlzhevE2PHH3OtP', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--166ab1f5-3287-4042-a225-0820b944b7f5-0' tool_calls=[{'name': 'plot_beta_distribution', 'args': {'alpha': 2, 'beta_param': 5}, 'id': 'call_IMjECwvWVDox7UyCHJu30SHB', 'type': 'tool_call'}] usage_metadata={'input_tokens': 145, 'output_tokens': 20, 'total_tokens': 165, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [48]:
print(ai_message.additional_kwargs)

{'refusal': None}
